# Amortizacion Francesa

## Configuración

Se importa `QC_Financial_3` que es la versión compliada para Python3.

In [1]:
import QC_Financial_3 as Qcf

### Otras Librerías

In [2]:
import pandas as pd

### Para Formato de `DataFrame`

In [9]:
format_dict = {'nominal': '{0:,.2f}', 'amortizacion': '{0:,.2f}', 'interes': '{0:,.2f}', 'flujo': '{0:,.2f}',
               'valor_tasa': '{0:,.4%}'}

## Legs
Los objetos de tipo `Leg` son una lista (o vector) de objetos `Cashflow` y representan una pata de un instrumento financiero. un objeto de tipo `Leg` puede construirse *a mano* es decir, dando de alta cashflows y agregándolos uno a uno o con algunos métodos de conveniencia cuyo funcionamiento se mostrará.

### Construcción Asistida de un `FrenchFixedRateLeg2`
Las patas con amortización francesa o cuotas iguales son las que habitualmente se usan en créditos de consumo e hipotecarios.

Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal inicial
- `bool`: si es `True` significa que la amortización es un flujo de caja efectivo
- `QCInterestRate`: la tasa a aplicar en cada flujo
- `QCCurrency`: moneda del nominal y de los flujos
- `bool`: si es `True` fuerza a que las fechas de pago coincidan con las fechas finales. Esto para lograr una valorización acorde a las convenciones de los mercados de renta fija, en caso que la `Leg` represente un bono a tasa fija.

Vamos a un ejemplo. Cambiando los parámetros siguientes se puede visualizar el efecto de ellos en la construcción.

In [4]:
# Se da de alta los parámetros requeridos
rp = Qcf.RecPay.RECEIVE
fecha_inicio = Qcf.QCDate(4, 4, 2018)
fecha_final = Qcf.QCDate(4, 4, 2020) 
bus_adj_rule = Qcf.BusyAdjRules.NO
periodicidad = Qcf.Tenor('1M')

# Con período irregular las cuotas no quedan iguales
periodo_irregular = Qcf.StubPeriod.NO

calendario = Qcf.BusinessCalendar(fecha_inicio, 20)
calendario.add_holiday(Qcf.QCDate(31, 12, 2018))
lag_pago = 0
nominal = 100000.0
amort_es_flujo = True

# Si la tasa es Act/360 o Act/365 las cuotas no van a quedar iguales
tasa_cupon = Qcf.QCInterestRate(.03, Qcf.QC30360(), Qcf.QCLinearWf())
moneda = Qcf.QCCLF()
es_bono = False

In [5]:
# Se da de alta el objeto
fixed_rate_leg_2 = Qcf.LegFactory.build_french_fixed_rate_leg_2(rp,
                                                                fecha_inicio,
                                                                fecha_final,
                                                                bus_adj_rule,
                                                                periodicidad,
                                                                periodo_irregular,
                                                                calendario,
                                                                lag_pago,
                                                                nominal,
                                                                amort_es_flujo,
                                                                tasa_cupon,
                                                                moneda,
                                                                es_bono)

In [8]:
tabla = []
columnas = list(Qcf.get_column_names('FixedRateCashflow2', ''))
for i in range(fixed_rate_leg_2.size()):
    tabla.append(Qcf.show(fixed_rate_leg_2.get_cashflow_at(i)))

df2 = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df2.style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2018-04-04,2018-05-04,2018-05-04,"100,000.00","4,048.12",250.00,True,"4,298.12",CLF,3.0000%,Lin30360
1,2018-05-04,2018-06-04,2018-06-04,"95,951.88","4,058.24",239.88,True,"4,298.12",CLF,3.0000%,Lin30360
2,2018-06-04,2018-07-04,2018-07-04,"91,893.64","4,068.39",229.73,True,"4,298.12",CLF,3.0000%,Lin30360
3,2018-07-04,2018-08-04,2018-08-06,"87,825.25","4,078.56",219.56,True,"4,298.12",CLF,3.0000%,Lin30360
4,2018-08-04,2018-09-04,2018-09-04,"83,746.69","4,088.75",209.37,True,"4,298.12",CLF,3.0000%,Lin30360
5,2018-09-04,2018-10-04,2018-10-04,"79,657.94","4,098.98",199.14,True,"4,298.12",CLF,3.0000%,Lin30360
6,2018-10-04,2018-11-04,2018-11-05,"75,558.96","4,109.22",188.90,True,"4,298.12",CLF,3.0000%,Lin30360
7,2018-11-04,2018-12-04,2018-12-04,"71,449.74","4,119.50",178.62,True,"4,298.12",CLF,3.0000%,Lin30360
8,2018-12-04,2019-01-04,2019-01-04,"67,330.24","4,129.80",168.33,True,"4,298.12",CLF,3.0000%,Lin30360
9,2019-01-04,2019-02-04,2019-02-04,"63,200.45","4,140.12",158.00,True,"4,298.12",CLF,3.0000%,Lin30360
